## Building A Chatbot

This chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [3]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key



'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'

In [6]:


from langchain_groq import ChatGroq
model = ChatGroq(model = "Gemma2-9b-It",groq_api_key = groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000271146D1340>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002711475A900>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [8]:
from langchain_core.messages import HumanMessage

result = model.invoke([HumanMessage(content="Hi, My name is nikhil and i am a data scientist")])
result

AIMessage(content="Hello Nikhil, it's nice to meet you!\n\nAs a large language model, I'm always interested in learning about people who work in data science. What kind of work do you do? Are you working on any interesting projects right now?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 22, 'total_tokens': 77, 'completion_time': 0.1, 'prompt_time': 0.002140346, 'queue_time': 0.23207518300000002, 'total_time': 0.102140346}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-0babfae3-54c5-4d6f-af88-1cfcb4677cf9-0', usage_metadata={'input_tokens': 22, 'output_tokens': 55, 'total_tokens': 77})

In [12]:
print(result.content)

Hello Nikhil, it's nice to meet you!

As a large language model, I'm always interested in learning about people who work in data science. What kind of work do you do? Are you working on any interesting projects right now?



In [13]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Nikhil and I am a Data scientist"),
        AIMessage(content="Hello Nikhil, it's nice to meet you! As a large language model, I'm always interested in learning about people who work in data science. What kind of work do you do? Are you working on any interesting projects right now?"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)



AIMessage(content="You said your name is Nikhil and that you are a data scientist. \n\nIs there anything else you'd like to tell me about yourself or your work? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 95, 'total_tokens': 135, 'completion_time': 0.072727273, 'prompt_time': 0.005296461, 'queue_time': 0.232773047, 'total_time': 0.078023734}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b37c4ca7-055d-423d-b1bc-9402cc46b908-0', usage_metadata={'input_tokens': 95, 'output_tokens': 40, 'total_tokens': 135})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [14]:
# !pip install langchain_community

In [15]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)




In [16]:
config={"configurable":{"session_id":"chat1"}}

In [17]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Nikhil and I am a data scientist")],
    config=config
)

In [18]:
response.content

"Hi Nikhil! It's nice to meet you.  \n\nThat's awesome that you're a data scientist! What kind of work do you do? Are you working on any interesting projects right now?  \n\nI'm always eager to learn more about what data scientists are up to. 😊\n"

In [19]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Nikhil.  \n\nI remember that from our earlier conversation! 😄  How can I help you today, Nikhil?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 102, 'total_tokens': 133, 'completion_time': 0.056363636, 'prompt_time': 0.00551606, 'queue_time': 0.233471329, 'total_time': 0.061879696}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-33d97c37-ebaf-4fcc-8926-06093d77031f-0', usage_metadata={'input_tokens': 102, 'output_tokens': 31, 'total_tokens': 133})

In [20]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to know! 😊\n"

In [21]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

'Nice to meet you, John! 👋  Is there anything I can help you with today?  😄 \n\n'

In [22]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'You told me your name is John!  😊 \n\nDid you forget?  😅\n'

In [23]:
store

{'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi , My name is Nikhil and I am a data scientist', additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Nikhil! It's nice to meet you.  \n\nThat's awesome that you're a data scientist! What kind of work do you do? Are you working on any interesting projects right now?  \n\nI'm always eager to learn more about what data scientists are up to. 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 22, 'total_tokens': 88, 'completion_time': 0.12, 'prompt_time': 0.002119396, 'queue_time': 0.231019753, 'total_time': 0.122119396}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-64fd27da-63be-4089-84fe-c5d9ccba4295-0', usage_metadata={'input_tokens': 22, 'output_tokens': 66, 'total_tokens': 88}), HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}), AIMessage(c

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [24]:

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt|model

Creates a Chat Prompt
ChatPromptTemplate.from_messages([...]) builds a conversation-style prompt that can accept system messages and dynamic chat history.

Defines the Assistant's Role
The system message tells the model to behave like a helpful assistant that answers questions to the best of its ability.

Uses MessagesPlaceholder
This allows you to insert a list of past chat messages (both human and AI) when invoking the chain — useful for maintaining conversation context.

Connects Prompt to Model
chain = prompt | model combines the prompt with your language model into a pipeline that takes input → formats prompt → gets model output.

Supports Dynamic Input
When you run chain.invoke({"messages": [...]}), it fills in the placeholder with the chat history and returns the assistant’s response.

In [ ]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Nikhil")]})

AIMessage(content="Hi Krish, it's nice to meet you! 👋\n\nHow can I help you today? 😊 \n", response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 36, 'total_tokens': 61, 'completion_time': 0.05, 'prompt_time': 0.003914296, 'queue_time': None, 'total_time': 0.053914296}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_e238d2cf52', 'finish_reason': 'stop', 'logprobs': None}, id='run-1f9187c3-4e8b-4ea5-809f-5c9d06d382dd-0', usage_metadata={'input_tokens': 36, 'output_tokens': 25, 'total_tokens': 61})

In [26]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [27]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Nikhil")],
    config=config
)

response

AIMessage(content="Hello Nikhil, it's nice to meet you!\n\nI'm ready to answer your questions to the best of my ability. Just ask away! 😄  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 32, 'total_tokens': 70, 'completion_time': 0.069090909, 'prompt_time': 0.003379827, 'queue_time': 0.243313111, 'total_time': 0.072470736}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-79316065-b689-4279-b0c8-49aa8c042dd5-0', usage_metadata={'input_tokens': 32, 'output_tokens': 38, 'total_tokens': 70})

In [28]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Nikhil, you told me!  😊  \n\n\nIs there anything else I can help you with?\n'

In [29]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [30]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Nikhil")],"language":"Hindi"})
response.content

'नमस्ते निक्खिल, \n\nमुझे बहुत खुशी हो कि आप मुझसे बात कर रहे हैं।  \n\nक्या आप मुझसे कुछ पूछना चाहते हैं? मैं आपकी मदद करने के लिए तैयार हूँ। \n\n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [31]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [32]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Nikhil")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते निखिल! 😊 \n\nमैं आपकी मदद करने के लिए यहाँ हूँ। आप मुझसे जो भी पूछना चाहें, पूछने में संकोच न करें।  \n\nक्या मैं आपके लिए कुछ कर सकता हूँ? \n'

In [33]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [31]:
response.content

'आपका नाम कृष है। 😊 \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [34]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

d:\ALL_projects_one_place\GenAI\GenAI_Project\projects\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\ALL_projects_one_place\GenAI\GenAI_Project\projects\myenv\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hp\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrat

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [36]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal information, including your favorite ice cream flavor!  \n\nWhat's your favorite ice cream? 😊  🍦\n"

In [37]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what is 2 + 2". 😊  \n\n\n\nLet me know if you want to try another one!\n'

In [38]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [39]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I don't have access to past conversations or personal information about you, so I don't know your name. Would you like to tell me? 😊  \n"

In [40]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  What is it? \n\n"